In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC

In [2]:
data = pd.read_csv('naukri.csv')

In [3]:
data = data.dropna()

In [4]:
data.reset_index(drop=True)

,Uniq Id,Job Title,Job Salary,Job Experience Required,Key Skills,Location,Industry,Role
0,9be62c49a0b7ebe982a4af1edaa7bc5f,Digital Media Planner,Not Disclosed by Recruiter,5 - 10 yrs,Media Planning| Digital Media,Mumbai,"Advertising, PR, MR, Event Management",Media Planning Executive/Manager
1,3c52d436e39f596b22519da2612f6a56,Online Bidding Executive,Not Disclosed by Recruiter,2 - 5 yrs,pre sales| closing| software knowledge| clien...,"Pune,Pune","IT-Software, Software Services",Sales Executive/Officer
2,ffad8a2396c60be2bf6d0e2ff47c58d4,Trainee Research/ Research Executive- Hi- Tec...,Not Disclosed by Recruiter,0 - 1 yrs,Computer science| Fabrication| Quality check|...,Gurgaon,"Recruitment, Staffing",R&D Executive
3,7b921f51b5c2fb862b4a5f7a54c37f75,Technical Support,"2,00,000 - 4,00,000 PA.",0 - 5 yrs,Technical Support,Mumbai,"IT-Software, Software Services",Technical Support Engineer
4,2d8b7d44e138a54d5dc841163138de50,Software Test Engineer -hyderabad,Not Disclosed by Recruiter,2 - 5 yrs,manual testing| test engineering| test cases|...,Hyderabad,"IT-Software, Software Services",Testing Engineer
...,...,...,...,...,...,...,...,...
28362,3510f2146b514f112eaa9706a16b3900,MTS-Software-Engineer,Not Disclosed by Recruiter,9 - 12 yrs,Graphics| C++| USB| Project management| SOC| ...,Bengaluru,"Semiconductors, Electronics",Software Developer
28363,4af8c56f5b4c535b1974065003f2d548,Client Relationship Manager,Not Disclosed by Recruiter,1 - 5 yrs,Service delivery| LMS| CRE| Integration servi...,Chennai,"Education, Teaching, Training",Operations Manager
28364,43d5b806b26ed99d3ebfc8f366b4cdd1,Assoc. Prof.- CS,Not Disclosed by Recruiter,4 - 6 yrs,Counselor| Mentor| Trainer| Advisor| Teaching,Howrah,"Education, Teaching, Training",Lecturer/Professor
28365,8bc87d14cf56564f16b6d4bae80180be,Application/Cyber Security Specialist-Pune,Not Disclosed by Recruiter,5 - 10 yrs,Security Analysis| Software Development Life ...,Pune,"IT-Software, Software Services",Software Developer


In [5]:
data['Job Title'] = data['Job Title'].fillna('')

In [6]:
data['Job Experience Required'] = data['Job Experience Required'].fillna('')

In [8]:
data['Key Skills'] = data['Key Skills'].fillna('')

In [9]:
data['Location'] = data['Location'].fillna('')

In [12]:
data['Industry'] = data['Industry'].fillna('')

In [11]:
data['Role'] = data['Role'].fillna('')

In [13]:
import re
data['Key Skills'] = data['Key Skills'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '', x)) # remove special characters
data['Location'] = data['Location'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '', x)) # remove special characters
data['Industry'] = data['Industry'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))
data['Role'] = data['Role'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))

data['Key Skills'] = data['Key Skills'].apply(lambda x: x.lower()) # convert to lowercase
data['Location'] = data['Location'].apply(lambda x: x.lower())
data['Industry'] = data['Industry'].apply(lambda x: x.lower())
data['Role'] = data['Role'].apply(lambda x: x.lower())

In [23]:
X = data[['Key Skills','Location', 'Industry', 'Role']]
y = data['Job Title']


In [32]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=100)
X_tfidf = vectorizer.fit_transform(X.apply(lambda x: ' '.join(x), axis=1))

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [30]:
svm = LinearSVC(random_state=42)

In [34]:
svm.fit(X_train, y_train)

LinearSVC(random_state=42)

In [35]:
y_pred = svm.predict(X_test)

In [36]:
accuracy = svm.score(X_test, y_test)
print('Accuracy:', accuracy)

Accuracy: 0.038597109622841026


In [37]:
from sklearn.metrics import f1_score
f1score = f1_score(y_test, y_pred, average='weighted')
print(f'F1-score: {f1score}')

F1-score: 0.019788087356212812


In [41]:
new_soundbite = pd.DataFrame({'Key Skills': ['media planning'],
                              'Location': ['mumbai'],
                              'Industry': ['advertising'],
                              'Role': ['manager']})
new_soundbite_tfidf = vectorizer.transform(new_soundbite.apply(lambda x: ' '.join(x), axis=1))
predictions = svm.decision_function(new_soundbite_tfidf)[0]
top_5_careers = predictions.argsort()[::-1][:5]
print('Top 5 careers:', [svm.classes_[i] for i in top_5_careers])

Top 5 careers: [' Production Manager', ' INTERN', ' Digital Media Planner', ' Social Media Manager (Digital Media)', ' Product/Brand Manager']
